# plot myeloid fine typing results

## wishlist:

- all correlation plots, all samples
- correlation heatmap
- plots in space

In [ ]:
require(Seurat)
require(tidyverse)
require(readxl)
require(patchwork)
require(sf)
require(ggpubr)
require(ggthemes)
require(harmony)
require(presto)
require(ComplexHeatmap)
require(circlize)
require(glue)
require(e1071) 
require(caTools) 
require(class) 
require(gghighlight)
require(tidyverse)
require(Seurat)
require(data.table)
require(lme4)
require(presto)
require(singlecellmethods)
require(future)
require(furrr)
require(gghighlight)
require(readr)
options(future.globals.maxSize = 1000 * 1024 ^2)
require(gridExtra)
set.seed(1)
options(repr.plot.res=300)
options(repr.matrix.max.cols=50, repr.matrix.max.rows=10000)

In [ ]:
source('/n/scratch/users/m/mup728/mup728/Cell_Typing_CRC_MERFISH/Fine_typing_with_weighted_KNN/Strom/fine_typing_libraries.r')

loadLibraries()
set.seed(1)

scRNA_Myeloid = readr::read_rds('/n/scratch/users/m/mup728/mup728/Cell_Typing_CRC_MERFISH/Pelka_reference_cleaning/Myeloid_fineTyping_selected_genes.rds')
scRNA_Myeloid@meta.data$technology = 'scRNA'
scRNA_Myeloid@meta.data$combined_cell_names = colnames(scRNA_Myeloid)

cellType = 'Myeloid'

selectedGenes = rownames(scRNA_Myeloid)

coarseTypedSeurats = list.files('/n/scratch/users/m/mup728/mup728/Pelka_Baysor_segmentation/coarse_typing_with_weighted_knn/Coarse_typing_with_weighted_knn/MSI/')[grep(x = list.files('/n/scratch/users/m/mup728/mup728/Pelka_Baysor_segmentation/coarse_typing_with_weighted_knn/Coarse_typing_with_weighted_knn/MSI/'), pattern = "coarse.*.rds")]

sampleIDs = sample(gsub(x = coarseTypedSeurats, pattern = 'coarse_|.rds', replacement = ""))


## correlation plots, all samples

In [ ]:
plotLists = paste('logFC_plotlist_', sampleIDs, '.rds', sep="")
plotLists

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 10, repr.plot.res = 300)
for (pl in plotLists){
    plotlist = read_rds(pl)
    ggsave(
       filename = gsub(x = pl, pattern = '.rds', replacement = '.pdf'), 
       plot = marrangeGrob(plotlist, nrow=1, ncol=1), 
       width = 15, height = 10
    )
}

## correlation heatmap

In [ ]:
# read in scRNA glmm
# read in merfish glmm
# calculate correlation coeff all genes
# calculate corr coeff selected genes
# store these variables in two dataframes

### note: some genes in merfish are not present in scRNA

In [ ]:
sampleID = sampleIDs[1]
scRNA_marginal_effects = read.csv(glue('effects_marginal_scRNA_', sampleID, '.csv'), row.names = 1) %>%
    mutate(technology = 'scRNA')
merfish_marginal_effects = read.csv(glue('effects_marginal_merfish_', sampleID, '.csv'), row.names = 1) %>%
    mutate(technology = 'MERFISH')
left_join(merfish_marginal_effects %>% select(contrast, feature, logFC, technology), scRNA_marginal_effects %>% select(contrast, feature, logFC, technology), join_by(contrast, feature), suffix = c('_MERFISH', '_scRNA')) %>% filter(is.na(logFC_scRNA)) %>% select(feature) %>% distinct() %>% as.matrix() %>% as.vector() %>% length() #%>% group_by(contrast) %>% na.omit() %>% summarize(corr = cor(logFC_MERFISH, logFC_scRNA))

### here we join the marginal effects and calculate correlations

In [ ]:
correlations = data.frame()
correlations

In [ ]:
for (sampleID in sampleIDs){
    scRNA_marginal_effects = read.csv(glue('effects_marginal_scRNA_', sampleID, '.csv'), row.names = 1) %>%
        mutate(technology = 'scRNA')
    merfish_marginal_effects = read.csv(glue('effects_marginal_merfish_', sampleID, '.csv'), row.names = 1) %>%
        mutate(technology = 'MERFISH')
    correlations = rbind(correlations, left_join(merfish_marginal_effects %>% select(contrast, feature, logFC, technology), scRNA_marginal_effects %>% select(contrast, feature, logFC, technology), join_by(contrast, feature), suffix = c('_MERFISH', '_scRNA')) %>% filter(!is.na(logFC_scRNA)) %>% group_by(contrast) %>% na.omit() %>% summarize(corr = cor(logFC_MERFISH, logFC_scRNA)) %>% mutate(sampleID = sampleID))
}

In [ ]:
temp = correlations %>% pivot_wider(values_from = corr, names_from = sampleID) %>% as.matrix()
options(repr.plot.res=300, repr.plot.height = 15, repr.plot.width = 15)
rownames(temp) = temp[,'contrast']
temp = temp[,2:ncol(temp)]
rowOrder = hclust(dist(temp, 
                       method = "euclidean"), 
                  method = "ward.D" )$order
colOrder = hclust(dist(t(temp), 
                       method = "euclidean"), 
                  method = "ward.D" )$order
temp = temp[rowOrder, colOrder] %>% as.data.frame() %>% mutate(contrast = rownames(.))
temp %>% as.data.frame() %>% pivot_longer(cols = colnames(.)[colnames(.) != 'contrast']) %>% rename('sampleID' = name, 'corr' = value) %>% mutate(corr = as.numeric(corr)) %>% 
ggplot(aes(y = contrast, x = sampleID)) +
geom_tile(aes(fill = corr)) +
geom_label(aes(label = round(corr, 1)), 
           color = 'red') +
theme_minimal(base_size = 16) +
theme(axis.text.x = element_text(angle = 90, 
                                 vjust = 0.5, 
                                 hjust=1)) +
scale_fill_viridis_c(option = 'inferno', na.value = 'white', direction = -1) + #, limits = c(0, 1)) 
ggtitle('Myeloid: all MERFISH genes')

In [ ]:
temp[is.na(temp)] = 0
temp = temp %>% select(!contrast) %>% as.matrix() %>% apply(MARGIN = c(1,2), FUN = as.numeric)

Heatmap(temp, name = 'Correlation', column_km = 2, row_km = 4, cell_fun = function(j, i, x, y, w, h, col) { # add text to each grid
        grid.text(round(temp[i, j],1), x, y)
    }, column_title = 'Myeloid: all MERFISH genes',)

### correlations with selected genes only

In [ ]:
selectedGenes

In [ ]:
correlations_selectedGenes = data.frame()
correlations_selectedGenes

In [ ]:
for (sampleID in sampleIDs){
    scRNA_marginal_effects = read.csv(glue('effects_marginal_scRNA_', sampleID, '.csv'), row.names = 1) %>%
        mutate(technology = 'scRNA') %>% filter(feature %in% selectedGenes)
    merfish_marginal_effects = read.csv(glue('effects_marginal_merfish_', sampleID, '.csv'), row.names = 1) %>%
        mutate(technology = 'MERFISH') %>% filter(feature %in% selectedGenes)
    correlations_selectedGenes = rbind(correlations_selectedGenes, left_join(merfish_marginal_effects %>% select(contrast, feature, logFC, technology), scRNA_marginal_effects %>% select(contrast, feature, logFC, technology), join_by(contrast, feature), suffix = c('_MERFISH', '_scRNA')) %>% filter(!is.na(logFC_scRNA)) %>% group_by(contrast) %>% na.omit() %>% summarize(corr = cor(logFC_MERFISH, logFC_scRNA)) %>% mutate(sampleID = sampleID))
}

In [ ]:
temp = correlations_selectedGenes %>% filter() %>% pivot_wider(values_from = corr, names_from = sampleID) %>% as.matrix()
rownames(temp) = temp[,'contrast']
temp = temp[,2:ncol(temp)]
rowOrder = hclust(dist(temp, 
                       method = "euclidean"), 
                  method = "ward.D" )$order
colOrder = hclust(dist(t(temp), 
                       method = "euclidean"), 
                  method = "ward.D" )$order
temp = temp[rowOrder, colOrder] %>% as.data.frame() %>% mutate(contrast = rownames(.))
temp %>% as.data.frame() %>% pivot_longer(cols = colnames(.)[colnames(.) != 'contrast']) %>% rename('sampleID' = name, 'corr' = value) %>% mutate(corr = as.numeric(corr)) %>% 
ggplot(aes(y = contrast, x = sampleID)) +
geom_tile(aes(fill = corr)) +
geom_label(aes(label = round(corr, 1)), 
           color = 'red') +
theme_minimal(base_size = 16) +
theme(axis.text.x = element_text(angle = 90, 
                                 vjust = 0.5, 
                                 hjust=1)) +
scale_fill_viridis_c(option = 'inferno', na.value = 'white', direction = -1) + 
ggtitle('Myeloid: selected genes') #, limits = c(0, 1))  


In [ ]:
temp[is.na(temp)] = 0
temp = temp %>% select(!contrast) %>% as.matrix() %>% apply(MARGIN = c(1,2), FUN = as.numeric)
Heatmap(temp, name = 'Correlation', column_km = 2, row_km = 4, cell_fun = function(j, i, x, y, w, h, col) { # add text to each grid
        grid.text(round(temp[i, j],1), x, y)
    }, column_title = 'Myeloid: selected genes',)

### tabulate numbers of cells per type per sample

In [ ]:
cell_numbers = data.frame()
cell_percent = data.frame()

In [ ]:
for (sampleID in sampleIDs){
    merged_Myeloid = read_rds(glue('Strom_fine_types_', sampleID, '.rds'))
    cell_percent = rbind(cell_percent, table(merged_Myeloid@meta.data$cleaned_fine_types, merged_Myeloid@meta.data$technology) %>% 
                         as.data.frame() %>% 
                         filter(Freq > 0) %>% 
                         rename('cleaned_fine_type' = Var1, 
                                'technology' = Var2) %>% 
                         pivot_wider(values_from = Freq, 
                                     names_from = technology) %>%
                         na.omit() %>% 
                         mutate(MERFISH = round(100 * na.omit(.$MERFISH)/sum(na.omit(.$MERFISH)), 2), 
                                scRNA = round(100 * na.omit(.$scRNA)/sum(na.omit(.$scRNA)), 2)) %>% 
                         mutate(sampleID = sampleID))
    cell_numbers = rbind(cell_numbers, 
                         table(merged_Myeloid@meta.data$cleaned_fine_types, 
                               merged_Myeloid@meta.data$technology) %>% 
                         as.data.frame() %>% 
                         filter(Freq > 0) %>% 
                         rename('cleaned_fine_type' = Var1, 
                                'technology' = Var2) %>% 
                         pivot_wider(values_from = Freq, 
                                     names_from = technology) %>% 
                         mutate(sampleID = sampleID))

}

In [ ]:
cell_numbers[is.na(cell_numbers)] = 0

In [ ]:
cell_numbers

In [ ]:
cell_percent 

In [ ]:
(cell_percent %>% filter(is.na(MERFISH)))
(cell_numbers %>% filter(is.na(MERFISH)))

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 15, repr.plot.res = 300)
cell_numbers %>% 
ggplot() + 
geom_col(aes(y = sampleID, x = MERFISH, fill = cleaned_fine_type)) +
scale_fill_tableau('Tableau 20') +
theme_minimal(base_size = 12) +
scale_x_continuous(trans = 'log10') +
facet_wrap(~cleaned_fine_type)

## plots in space

In [ ]:
for (sampleID in sampleIDs){
    merged_Myeloid = read_rds(glue('Myeloid_fine_types_', sampleID, '.rds'))
    coarse_types = readr::read_rds(glue('/n/scratch/users/m/mup728/mup728/Pelka_Baysor_segmentation/coarse_typing_with_weighted_knn/Coarse_typing_with_weighted_knn/MSI/coarse_',
                                        sampleID,
                                        '.rds'))
    p1 = ggplot(merged_Myeloid@meta.data %>% filter(technology == 'MERFISH')) +
    geom_point(shape = '.', aes(x = x, y = y, color = cleaned_fine_types)) +
    ggthemes::scale_color_tableau('Tableau 20') +
    theme_minimal(base_size = 18) +
    guides(colour = guide_legend(override.aes = list(size=10, shape=16))) +
    coord_sf() 

    constantPalette = rep(x = 'red', times = length(unique(coarse_types@meta.data$ClusterTop)) +
                                                   length(unique(merged_Myeloid@meta.data$cleaned_fine_types)))

    p2 = full_join(coarse_types@meta.data %>% filter(technology == 'MERFISH') %>% select(x, y, ClusterTop),
             merged_Myeloid@meta.data %>% filter(technology == 'MERFISH') %>% select(x, y, cleaned_fine_types),
             by = join_by(ClusterTop == cleaned_fine_types, x, y)) %>%
    rename('Clusters' = ClusterTop) %>%
    ggplot() + 
    geom_point(shape = '.', aes(x = x, y = y), color = 'red') +
    theme_minimal(base_size=18) +
    scale_color_manual(values = constantPalette) +
    coord_sf() +
    facet_wrap(~Clusters) +
    gghighlight()

    p3 = coarse_types@meta.data %>% filter(technology == 'MERFISH') %>% select(x, y, ClusterTop) %>%
        ggplot() + 
        geom_point(shape = '.', aes(x = x, y = y, color = ClusterTop)) +
        theme_minimal(base_size=18) +
        scale_color_tableau('Color Blind') +
        coord_sf() +
        guides(colour = guide_legend(override.aes = list(size=10, shape=16))) 

    p4 = coarse_types@meta.data %>% filter(technology == 'MERFISH') %>% select(x, y, ClusterTop) %>%
        ggplot() + 
        geom_point(shape = '.', aes(x = x, y = y, color = ClusterTop)) +
        theme_minimal(base_size=18) +
        scale_color_tableau('Color Blind') +
        coord_sf() +
        facet_wrap(~ClusterTop) +
        gghighlight()

    plotList = list(p1, p2, p3, p4)
    ggsave(
       filename = glue(sampleID, "_Myeloid_spatial_plots.pdf"), 
       plot = marrangeGrob(plotList, nrow=1, ncol=1), 
       width = 20, height = 20
    )
}